In [ ]:
# vtu_pipeline_final_fixed.py
import pandas as pd
import numpy as np
import random
from datetime import date, timedelta
from pathlib import Path
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, balanced_accuracy_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import joblib
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# Version‑proof OneHotEncoder
def make_ohe():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)  # sklearn >= 1.2
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)         # sklearn <= 1.1

# Try XGBoost; fallback to RF
USE_XGB = True
try:
    from xgboost import XGBClassifier
except Exception:
    USE_XGB = False

# Try SHAP; fallback to feature importance plot
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

# -----------------------------
# Config
# -----------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

DEPARTMENTS = ["CSE", "ISE", "ECE", "EEE", "MECH"]
N_STUDENTS_PER_DEPT = 120
SEMESTER = 5
WEEKS = 16
SEM_START = date(2025, 8, 1)
WEEK_STARTS = [SEM_START + timedelta(weeks=i) for i in range(WEEKS)]
FESTIVAL_WEEKS = {date(2025, 11, 3)}  # e.g., Diwali week

PASS_MARK = 50
BASELINE_PASS_RATE = 0.70  # 70%

DEPT_COURSES = {
    "CSE": [
        ("Software Engineering", 4, "Theory"),
        ("Computer Networks", 4, "Theory"),
        ("DBMS", 4, "Theory"),
        ("Automata Theory", 4, "Theory"),
        ("Machine Learning", 4, "Theory"),
        ("CN Lab", 2, "Lab"),
        ("DBMS Lab", 2, "Lab")
    ],
    "ISE": [
        ("Software Engineering", 4, "Theory"),
        ("Data Communication", 4, "Theory"),
        ("DBMS", 4, "Theory"),
        ("Automata Theory", 4, "Theory"),
        ("Data Mining", 4, "Theory"),
        ("CN Lab", 2, "Lab"),
        ("DBMS Lab", 2, "Lab")
    ],
    "ECE": [
        ("Digital Communication", 4, "Theory"),
        ("Microprocessors", 4, "Theory"),
        ("Control Systems", 4, "Theory"),
        ("VLSI Design", 4, "Theory"),
        ("Signals & Systems", 4, "Theory"),
        ("VLSI Lab", 2, "Lab"),
        ("DSP Lab", 2, "Lab")
    ],
    "EEE": [
        ("Power Systems", 4, "Theory"),
        ("Control Systems", 4, "Theory"),
        ("Microcontrollers", 4, "Theory"),
        ("Electrical Machines", 4, "Theory"),
        ("Power Electronics", 4, "Theory"),
        ("Machines Lab", 2, "Lab"),
        ("Power Electronics Lab", 2, "Lab")
    ],
    "MECH": [
        ("Dynamics of Machines", 4, "Theory"),
        ("Design of Machine Elements", 4, "Theory"),
        ("Heat Transfer", 4, "Theory"),
        ("Fluid Mechanics", 4, "Theory"),
        ("Manufacturing Process", 4, "Theory"),
        ("Thermal Lab", 2, "Lab"),
        ("FM Lab", 2, "Lab")
    ]
}

OUT = Path(".")
OUT.mkdir(parents=True, exist_ok=True)

# -----------------------------
# Helpers
# -----------------------------
def clipped_normal(mu, sigma, low, high, size=None):
    x = np.random.normal(mu, sigma, size)
    return np.clip(x, low, high)

def linear_trend(arr):
    x = np.arange(len(arr))
    if len(arr) < 2 or np.all(arr == arr[0]):
        return 0.0
    return float(np.polyfit(x, arr, 1)[0])

def choose_section():
    return random.choice(list("ABCDE"))

def choose_gender():
    return random.choice(["Male", "Female"])

# -----------------------------
# 1) Students & Courses
# -----------------------------
students, courses = [], []
cid = 1
for dept in DEPARTMENTS:
    for cname, credits, ctype in DEPT_COURSES[dept]:
        courses.append({
            "CourseID": f"C{cid:03d}",
            "CourseName": cname,
            "Department": dept,
            "Semester": SEMESTER,
            "Credits": credits,
            "CourseType": ctype
        })
        cid += 1

sid_counter = 1
for dept in DEPARTMENTS:
    for _ in range(N_STUDENTS_PER_DEPT):
        # Tri-modal latent ability for clearer bands
        ability_mode = np.random.choice(["low","mid","high"], p=[0.33,0.34,0.33])
        if ability_mode == "low":
            latent_ability = np.random.normal(0.35, 0.07)
        elif ability_mode == "mid":
            latent_ability = np.random.normal(0.55, 0.07)
        else:
            latent_ability = np.random.normal(0.78, 0.07)
        latent_ability = float(np.clip(latent_ability, 0.15, 0.95))

        students.append({
            "StudentID": f"S{sid_counter:04d}",
            "Department": dept,
            "Section": choose_section(),
            "Age": int(np.random.randint(18, 25)),
            "Gender": choose_gender(),
            "LatentAbility": latent_ability,
            "PriorGPA": round(np.clip(np.random.normal(6.8 + 2.2*latent_ability, 0.6), 5.0, 9.8), 1)
        })
        sid_counter += 1

students_df = pd.DataFrame(students)
courses_df = pd.DataFrame(courses)

# -----------------------------
# 2) Enrollments, Assessments, LMS
# -----------------------------
enrollments, assessments, lms_rows = [], [], []

for _, s in students_df.iterrows():
    sid = s["StudentID"]; dept = s["Department"]; ability = s["LatentAbility"]
    dept_courses = courses_df[courses_df["Department"] == dept]
    base_attendance = np.clip(60 + 30*ability + np.random.normal(0, 8), 35, 96)

    for _, c in dept_courses.iterrows():
        cid = c["CourseID"]; ctype = c["CourseType"]
        attendance = float(np.clip(np.random.normal(base_attendance, 6), 35, 96))
        assign_avg = float(np.clip(50 + 40*ability + 0.15*attendance + np.random.normal(0, 8), 20, 98))
        quiz_avg   = float(np.clip(48 + 40*ability + 0.12*attendance + np.random.normal(0, 10), 10, 98))
        ia1 = float(np.clip(0.35*attendance + 0.35*assign_avg + 0.30*quiz_avg + np.random.normal(0, 6), 0, 100))
        ia2 = float(np.clip(0.40*attendance + 0.30*assign_avg + 0.30*quiz_avg + np.random.normal(0, 6), 0, 100))
        ia3 = float(np.clip(0.40*attendance + 0.25*assign_avg + 0.35*quiz_avg + np.random.normal(0, 6), 0, 100))
        project = float(np.clip(55 + 45*ability + np.random.normal(0, 10), 20, 98)) if ctype=="Lab" else np.nan

        # LMS weekly activity with festival dip and late push
        for wk in WEEK_STARTS:
            outage = wk in FESTIVAL_WEEKS
            week_idx = (wk - WEEK_STARTS[0]).days // 7
            late_push = 1.0 + 0.02*max(0, week_idx - 8)
            base_lambda = (3 if ctype=="Theory" else 2) * (0.8 + 0.6*ability) * late_push
            logins = 0 if outage else np.random.poisson(max(0.5, base_lambda))
            time_mins = 0 if outage else max(0, np.random.normal((85 if ctype=="Theory" else 70)*(0.8+0.6*ability), 30))
            videos = 0 if outage else np.random.poisson(max(0.5, (3 if ctype=="Theory" else 2)*(0.8+0.6*ability)))
            posts = 0 if outage else np.random.poisson(max(0.3, 1*(0.8+0.4*ability)))

            lms_rows.append({
                "StudentID": sid, "CourseID": cid, "WeekStart": wk,
                "LMSLogins": int(logins), "TimeOnPlatformMins": float(round(time_mins,1)),
                "VideosWatched": int(videos), "ForumPosts": int(posts)
            })

        # Final score strongly tied to IA2/IA3 + CA + project (lab)
        final_score = (
            0.35*ia2 + 0.25*ia3 + 0.18*assign_avg + 0.10*quiz_avg +
            (0.12*project if ctype=="Lab" else 0) + np.random.normal(0, 6)
        )
        final_score = float(np.clip(final_score, 25, 98))

        enrollments.append({
            "StudentID": sid, "CourseID": cid,
            "AttendanceRate": round(attendance,1),
            "AssignmentScoreAvg": round(assign_avg,1),
            "QuizScoreAvg": round(quiz_avg,1),
            "ProjectScore": round(project,1) if not np.isnan(project) else np.nan
        })

        assessments.append({
            "StudentID": sid, "CourseID": cid,
            "IA1": round(ia1,1), "IA2": round(ia2,1), "IA3": round(ia3,1),
            "FinalExamScore": round(final_score,1)
        })

enrollments_df = pd.DataFrame(enrollments)
assessments_df = pd.DataFrame(assessments)
lms_df = pd.DataFrame(lms_rows)

# -----------------------------
# 3) Feature engineering (25+ features)
# -----------------------------
def aggregate_lms(lms: pd.DataFrame) -> pd.DataFrame:
    aggs = lms.groupby(["StudentID","CourseID"]).agg(
        logins_mean=("LMSLogins","mean"),
        logins_std=("LMSLogins","std"),
        time_mean=("TimeOnPlatformMins","mean"),
        time_std=("TimeOnPlatformMins","std"),
        videos_mean=("VideosWatched","mean"),
        posts_mean=("ForumPosts","mean"),
        weeks_active=("LMSLogins",lambda x: int((x>0).sum())),
        weeks_inactive=("LMSLogins",lambda x: int((x==0).sum()))
    ).reset_index()

    trends = []
    early_late = []
    for (sid, cid), grp in lms.groupby(["StudentID","CourseID"]):
        g = grp.sort_values("WeekStart")
        early = g.head(4)["LMSLogins"].sum()
        late  = g.tail(4)["LMSLogins"].sum()
        edrop = (early - late) / (early + 1e-6)
        trends.append({
            "StudentID": sid, "CourseID": cid,
            "logins_trend": linear_trend(g["LMSLogins"].values),
            "time_trend": linear_trend(g["TimeOnPlatformMins"].values)
        })
        early_late.append({
            "StudentID": sid, "CourseID": cid,
            "engagement_drop_rate": edrop
        })
    trends = pd.DataFrame(trends)
    early_late = pd.DataFrame(early_late)

    out = aggs.merge(trends, on=["StudentID","CourseID"], how="left") \
              .merge(early_late, on=["StudentID","CourseID"], how="left")
    out["engagement_index"] = (
        0.4*out["logins_mean"] +
        0.4*(out["time_mean"]/60.0) +
        0.1*out["videos_mean"] +
        0.1*out["posts_mean"]
    )
    return out

lms_agg = aggregate_lms(lms_df)

feat = enrollments_df.merge(assessments_df, on=["StudentID","CourseID"], how="left") \
                     .merge(lms_agg, on=["StudentID","CourseID"], how="left") \
                     .merge(courses_df[["CourseID","Department","Credits","CourseType","CourseName"]], on="CourseID", how="left") \
                     .merge(students_df[["StudentID","Section","Age","Gender","PriorGPA"]], on="StudentID", how="left")

feat["IA_avg"] = feat[["IA1","IA2","IA3"]].mean(axis=1)
feat["IA_trend"] = feat["IA3"] - feat["IA1"]
feat["attendance_flag_lt75"] = (feat["AttendanceRate"] < 75).astype(int)
feat["ia2_below60"] = (feat["IA2"] < 60).astype(int)
feat["quiz_below60"] = (feat["QuizScoreAvg"] < 60).astype(int)
feat["assign_below60"] = (feat["AssignmentScoreAvg"] < 60).astype(int)
feat["lab_weak"] = ((feat["CourseType"]=="Lab") & (feat["ProjectScore"] < 65)).astype(int)
feat["inactivity_ratio"] = feat["weeks_inactive"] / (feat["weeks_active"] + feat["weeks_inactive"]).replace(0,1)
feat["age_norm"] = (feat["Age"] - feat["Age"].mean())/feat["Age"].std(ddof=0)
feat["attendance_volatility"] = feat["logins_std"].fillna(0) / (feat["logins_mean"].fillna(0) + 1e-6)
feat["prior_coursework_score"] = np.clip(55 + 5*feat["PriorGPA"] + np.random.normal(0,6,len(feat)), 40, 98)
feat["lab_theory_gap"] = np.where(feat["CourseType"]=="Lab",
                                  feat["ProjectScore"] - feat["QuizScoreAvg"],
                                  feat["QuizScoreAvg"] - feat["AssignmentScoreAvg"])

feat["PerformanceBand"] = pd.cut(
    feat["FinalExamScore"],
    bins=[-0.1, 55, 75, 100],
    labels=["Low","Medium","High"]
)

# Save raw tables (strip latent ability)
students_export = students_df.drop(columns=["LatentAbility"])
students_export.to_csv("students.csv", index=False)
courses_df.to_csv("courses.csv", index=False)
enrollments_df.to_csv("enrollments.csv", index=False)
assessments_df.to_csv("assessments.csv", index=False)
lms_df.to_csv("lms.csv", index=False)
feat.to_csv("features.csv", index=False)

# -----------------------------
# 4) Modeling
# -----------------------------
numeric_cols = [
    "AttendanceRate","AssignmentScoreAvg","QuizScoreAvg","ProjectScore",
    "IA1","IA2","IA3","IA_avg","IA_trend",
    "logins_mean","logins_std","time_mean","time_std",
    "videos_mean","posts_mean","weeks_active","weeks_inactive",
    "logins_trend","time_trend","engagement_index","engagement_drop_rate",
    "inactivity_ratio","attendance_volatility","Credits","PriorGPA",
    "prior_coursework_score","lab_theory_gap","age_norm",
    "attendance_flag_lt75","ia2_below60","quiz_below60","assign_below60","lab_weak"
]
numeric_cols = list(dict.fromkeys(numeric_cols))
cat_cols = ["Department","CourseType","Section","Gender"]

model_df = feat.dropna(subset=["PerformanceBand"]).copy()
X = model_df[numeric_cols + cat_cols]
y = model_df["PerformanceBand"]

ohe = make_ohe()
ct = ColumnTransformer([
    ("num","passthrough", numeric_cols),
    ("cat", ohe, cat_cols)
])

if USE_XGB:
    clf = XGBClassifier(
        n_estimators=600,
        max_depth=6,
        learning_rate=0.06,
        subsample=0.9,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        reg_alpha=0.0,
        objective="multi:softprob",
        eval_metric="mlogloss",
        random_state=SEED,
        n_jobs=-1
    )
else:
    clf = RandomForestClassifier(
        n_estimators=900,
        max_depth=None,
        min_samples_leaf=2,
        class_weight="balanced_subsample",
        random_state=SEED,
        n_jobs=-1
    )

pipe = Pipeline([("prep", ct), ("clf", clf)])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("\nClass distribution (train):")
print(y_train.value_counts(normalize=True).round(3))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Balanced accuracy:", round(balanced_accuracy_score(y_test, y_pred), 4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

joblib.dump(pipe, "student_perf_clf.joblib")

# -----------------------------
# 5) Early Warning Alerts
# -----------------------------
proba_all = pipe.predict_proba(X)
classes = list(pipe.classes_)
p_low = proba_all[:, classes.index("Low")]

alerts_df = model_df[["StudentID","CourseID"]].copy()
alerts_df["PredictedBand"] = pipe.predict(X)
alerts_df["PLow"] = np.round(p_low, 3)

def risk_bucket(p):
    if p >= 0.60: return "High"
    if p >= 0.35: return "Medium"
    return "Low"

alerts_df["RiskLevel"] = alerts_df["PLow"].apply(risk_bucket)

# Assign alert dates skewed to mid/late term (after IA1)
alert_weeks = WEEK_STARTS[5:]  # from week 6 onwards
alerts_df["AlertDate"] = [random.choice(alert_weeks) for _ in range(len(alerts_df))]

# Attach drivers for reasons
subset = model_df[["StudentID","CourseID","AttendanceRate","IA2","QuizScoreAvg","engagement_index","engagement_drop_rate"]]
alerts_df = alerts_df.merge(subset, on=["StudentID","CourseID"], how="left")

def reason(row):
    r = []
    if row["AttendanceRate"] < 60: r.append("Low attendance")
    if row["IA2"] < 55: r.append("Weak IA2")
    if row["QuizScoreAvg"] < 55: r.append("Low quiz avg")
    if row["engagement_index"] < 3.0: r.append("Low LMS engagement")
    if row["engagement_drop_rate"] > 0.25: r.append("Engagement dropped")
    return ", ".join(r) if r else "Stable"

alerts_df["RiskReason"] = alerts_df.apply(reason, axis=1)

alerts_export = alerts_df[[
    "StudentID","CourseID","PredictedBand","PLow","RiskLevel","RiskReason","AlertDate"
]]
alerts_export.to_csv("alerts.csv", index=False)

# -----------------------------
# 6) Recommendations
# -----------------------------
def recommend(row):
    recs = []
    if row["AttendanceRate"] < 75:
        recs.append("Attendance counselling + buddy system")
    if row["IA2"] < 60:
        recs.append("IA2 remedial + topic-wise practice set")
    if row["QuizScoreAvg"] < 60:
        recs.append("Concept videos + micro-quizzes")
    if row["engagement_index"] < 3.0:
        recs.append("LMS study plan (3x/week) + tutor check-ins")
    if row["engagement_drop_rate"] > 0.25:
        recs.append("Motivation check-in + peer group project")
    if not recs:
        recs.append("Advanced project / enrichment tasks")
    return "; ".join(recs)

recs_df = subset.copy()
recs_df["RecommendedAction"] = recs_df.apply(recommend, axis=1)
recs_export = recs_df[["StudentID","CourseID","RecommendedAction"]]
recs_export.to_csv("recommendations.csv", index=False)

# -----------------------------
# 7) Explainability (global)
# -----------------------------
if SHAP_AVAILABLE and USE_XGB:
    # SHAP on transformed sample
    explainer = shap.TreeExplainer(pipe.named_steps["clf"])
    Xt_train = pipe.named_steps["prep"].fit_transform(X_train, y_train)
    Xt_sample = Xt_train[:min(3000, Xt_train.shape[0])]
    shap_values = explainer.shap_values(Xt_sample)
    shap.summary_plot(shap_values, Xt_sample, plot_type="bar", show=False)
    plt.tight_layout()
    plt.savefig("shap_summary_plot.png", dpi=200)
    plt.close()
else:
    # Feature importance bar chart
    model = pipe.named_steps["clf"]
    try:
        importances = model.feature_importances_
        # Cat feature names
        cat_names = list(ohe.get_feature_names_out(cat_cols))
        feat_names = numeric_cols + cat_names
        imp_df = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False).head(20)
        plt.figure(figsize=(8,6))
        plt.barh(imp_df["feature"][::-1], imp_df["importance"][::-1])
        plt.title("Top 20 Feature Importances")
        plt.tight_layout()
        plt.savefig("shap_summary_plot.png", dpi=200)
        plt.close()
    except Exception:
        plt.figure(figsize=(6,4))
        plt.text(0.5,0.5,"Importance plot unavailable", ha="center", va="center")
        plt.axis("off")
        plt.savefig("shap_summary_plot.png", dpi=200)
        plt.close()

# -----------------------------
# 8) Power BI-ready aggregates
# -----------------------------
# 8.1 Summary metrics
total_students = students_export["StudentID"].nunique()
high_risk_students = alerts_export.query("RiskLevel == 'High'")["StudentID"].nunique()
high_risk_pct = (high_risk_students / total_students) * 100.0
avg_attendance = enrollments_df["AttendanceRate"].mean()
# Pass rate: distinct students with max FinalExamScore >= PASS_MARK
passers = assessments_df.groupby("StudentID")["FinalExamScore"].max().ge(PASS_MARK).sum()
pass_rate_pct = (passers / total_students) * 100.0
retention_lift_pct = (pass_rate_pct/100.0 - BASELINE_PASS_RATE) * 100.0

pd.DataFrame([{
    "total_students": int(total_students),
    "high_risk_%": round(high_risk_pct, 1),
    "average_attendance": round(avg_attendance, 1),
    "pass_rate_%": round(pass_rate_pct, 1),
    "retention_lift_%": round(retention_lift_pct, 1)
}]).to_csv("summary_metrics.csv", index=False)

# 8.2 Risk trend (highest risk per student-week)
trend = alerts_export.copy()
trend["week"] = pd.to_datetime(trend["AlertDate"]).dt.to_period("W").apply(lambda r: r.start_time)
risk_priority = {"High": 3, "Medium": 2, "Low": 1}
trend["risk_score"] = trend["RiskLevel"].map(risk_priority)
trend_sorted = trend.sort_values(["StudentID","week","risk_score"], ascending=[True, True, False])
trend_consolidated = trend_sorted.drop_duplicates(subset=["StudentID","week"], keep="first")
risk_trend = trend_consolidated.groupby(["week","RiskLevel"])["StudentID"].nunique().reset_index()
risk_trend.columns = ["week","risk_level","count"]
risk_trend.to_csv("risk_trend.csv", index=False)

# 8.3 Risk by dept & course (avoid duplicate Department columns)
risk_by_dept_course = (
    alerts_export
      .merge(students_export[["StudentID","Department"]], on="StudentID", how="left")
      .merge(courses_df[["CourseID","CourseName"]], on="CourseID", how="left")
      .groupby(["Department","CourseName","RiskLevel"])["StudentID"].nunique()
      .reset_index()
      .rename(columns={
          "Department":"department",
          "CourseName":"course",
          "RiskLevel":"risk_level",
          "StudentID":"count"
      })
)
risk_by_dept_course.to_csv("risk_by_dept_course.csv", index=False)

# 8.4 Student 360 (avoid duplicate Department columns)
student_360 = (
    alerts_export
      .merge(students_export[["StudentID","Department"]], on="StudentID", how="left")
      .merge(courses_df[["CourseID","CourseName"]], on="CourseID", how="left")
      .merge(enrollments_df, on=["StudentID","CourseID"], how="left")
      .merge(assessments_df, on=["StudentID","CourseID"], how="left")
      .merge(recs_export, on=["StudentID","CourseID"], how="left")
      .loc[:, ["StudentID","Department","CourseName","AttendanceRate","IA1","IA2","IA3",
               "FinalExamScore","RiskLevel","RecommendedAction"]]
      .rename(columns={
          "StudentID":"student_id",
          "Department":"department",
          "CourseName":"course",
          "AttendanceRate":"attendance",
          "FinalExamScore":"final_score",
          "RiskLevel":"risk_level",
          "RecommendedAction":"recommended_action"
      })
)
student_360.to_csv("student_360.csv", index=False)

print("\nSaved files:")
print("- students.csv, courses.csv, enrollments.csv, assessments.csv, lms.csv")
print("- features.csv")
print("- alerts.csv")
print("- recommendations.csv")
print("- shap_summary_plot.png")
print("- student_perf_clf.joblib")
print("- summary_metrics.csv")
print("- risk_trend.csv")
print("- risk_by_dept_course.csv")
print("- student_360.csv")
